In [34]:
"""下面比较使用bert直接处理和间接处理两种方式处理句子
1.直接处理
"""
from transformers import BertTokenizer,BertModel
tokenizer = BertTokenizer.from_pretrained("/home/lawson/pretrain/bert-base-cased")
bert = BertModel.from_pretrained("/home/lawson/pretrain/bert-base-cased")
texts = "They threw a party for the inventor of the toaster . And he was toasted ."
inputs = tokenizer(texts,return_tensors = 'pt')
out = bert(**inputs)
a,b = out
print(a.size())
print(a[0,1,1]) # 随便找个数输出

torch.Size([1, 20, 768])
tensor(-0.4046, grad_fn=<SelectBackward>)


In [40]:
"""
2.间接处理。需要注意的事项：
（1）attention_mask, input_ids, token_type_ids 都必须是二维的 
（2）上述三者都必须是tensor
"""
temp = texts.split()
print(temp)

# 手动创建 inputs
allTokens = []

for word in temp:
    tokens  = tokenizer.tokenize(word)
    allTokens.extend(tokens)
print(len(allTokens))

input_ids = []
for token in allTokens:
    iid = tokenizer.convert_tokens_to_ids(token)
    input_ids.append(iid)
print(input_ids)
input_ids.insert(0,101)
input_ids.append(102)
print(len(input_ids))


import torch as t
attention_mask = [1]*20
token_type_ids = [0] * 20
input_ids = t.tensor(input_ids)
attention_mask = t.tensor(attention_mask)
token_type_ids = t.tensor(token_type_ids)

input_ids = input_ids.unsqueeze(0)
attention_mask = attention_mask.unsqueeze(0)
token_type_ids = token_type_ids.unsqueeze(0)
print(input_ids.size())

res = bert(input_ids = input_ids,
            attention_mask = attention_mask,
            token_type_ids = token_type_ids)
x = res[0]
print(x.size())
print(x[0,1,1])

['They', 'threw', 'a', 'party', 'for', 'the', 'inventor', 'of', 'the', 'toaster', '.', 'And', 'he', 'was', 'toasted', '.']
18
[1152, 3885, 170, 1710, 1111, 1103, 12989, 1104, 1103, 17458, 1200, 119, 1105, 1119, 1108, 17458, 1174, 119]
20
torch.Size([1, 20])
torch.Size([1, 20, 768])
tensor(-0.4046, grad_fn=<SelectBackward>)
